In [1]:
# Importing packages
import time, random
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

In [2]:
# SQL setup
from sqlalchemy import create_engine
# connect to a local database
engine = create_engine("sqlite:///pgastats.db")

In [3]:
url='https://www.pgatour.com/stats/stat.02674.y2022.eoff.t028.html' 
time.sleep(1+2*random.random())
response=requests.get(url)
time.sleep(1+2*random.random())
print(response.status_code)
html=response.text
soup=bs(html)

200


In [4]:
# Grab the statistic name
#stat_name=soup.find('section',class_='statistics-details-content').find('h1').text
stat_name=soup.find('div',class_='breadcrumbs').find('a',class_='current').text



In [5]:
# Grab the year
year=soup.find('div',class_='statistics-details-select-wrap season').find('option',selected='selected').text


In [6]:
# Grab the time period
time_period=soup.find('div',class_='statistics-details-select-wrap period').find('option',selected='selected').text

In [7]:
# Grab the tournament name
tournament_name=soup.find('div',class_='statistics-details-select-wrap tournament').find('option',selected='selected').text


In [8]:
# Grabs HTML of entire table

table=soup.find('div',class_='details-table-wrap')

In [9]:
# Grabs just the header of the table (columns)

table_head=table.find('thead')

In [10]:
# Grabs the column name indiviual tags

columns=table_head.find_all('th')

In [11]:
# Loop through column tags and save the column names to list
column_names=[]
for tag in columns:
    column_names.append(tag.text.strip())

In [12]:
# Grabs the body of the table (rows)

table_body=table.find('tbody')

In [13]:
# Grabs the row indiviual tags

rows=table_body.find_all('tr')

In [14]:
# Build a list of lists that contains the values from each row

row_list=[]
for tag in rows:
    values=tag.find_all('td')
    value_list=[]
    for value_tag in values:
        value_list.append(value_tag.text.strip())
    row_list.append(value_list)

In [15]:
row_dict_list=[]
for row in row_list:
    dictionary=dict(zip(column_names,row))
    row_dict_list.append(dictionary)

In [16]:
df=pd.DataFrame(row_dict_list)

In [17]:
df['Year']=year
df['Time Period']=time_period
df['Tournament']=tournament_name

In [18]:
df

,Year,Time Period,Tournament


In [19]:
df.to_sql('%s_%s_%s_%s'%(stat_name,year,time_period,tournament_name),engine,index=False)

0